In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from result_saver import SaverProvider

provider = SaverProvider()

In [6]:
DEVICE = 'ibm_sherbrooke'
DISTANCE = 10
ROUNDS = 10
_RESETS = False
LOGICAL = 0

In [7]:
from soft_info import find_longest_path_in_hex

backend = provider.get_backend(DEVICE)
longest_path, _, _ = find_longest_path_in_hex(backend)
print(longest_path)

[113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 111, 104, 103, 102, 101, 100, 99, 98, 97, 96, 95, 94, 90, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 73, 66, 65, 64, 63, 62, 61, 60, 59, 58, 57, 56, 52, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 35, 28, 27, 26, 25, 24, 23, 22, 21, 20, 19, 18, 14, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 17, 30, 31, 32, 36, 51, 50, 49, 55, 68, 69, 70, 74, 89, 88, 87, 93, 106, 107, 108, 112, 126, 125, 124, 123]


In [8]:
from qiskit_qec.circuits import RepetitionCodeCircuit
from qiskit import transpile
from tqdm import tqdm

circuits = []
for i in tqdm(range(len(longest_path)-(2*DISTANCE-1))):
    bounded_path = longest_path[i : i+2*DISTANCE-1]
    layout = bounded_path[1::2] + bounded_path[::2] 
    code = RepetitionCodeCircuit(DISTANCE, ROUNDS, xbasis=False, resets=_RESETS)
    qc = code.circuit[str(LOGICAL)]
    circuits.append(transpile(qc, backend, initial_layout=layout, optimization_level=1))

100%|██████████| 90/90 [00:13<00:00,  6.63it/s]


In [9]:
# get the rep_delay
rep_delay=backend.configuration().rep_delay_range[1]
print(rep_delay)

0.0005


In [12]:
SHOTS = 1e6 / (DISTANCE*ROUNDS) * 1
print('SHOTS', SHOTS)
print('SHOTS/len(circuits)', SHOTS/len(circuits))

SHOTS 10000.0
SHOTS/len(circuits) 111.11111111111111


In [13]:
from Scratch import metadata_helper


for i in range(10):
    metadata = metadata_helper(descr="Mean positions", code="RepetitionCodeCircuit", distance=DISTANCE,
                            rounds=ROUNDS, resets=_RESETS, logical=LOGICAL)
    backend.run(metadata, circuits, shots=SHOTS/len(circuits), meas_level=1, meas_return='single', 
                job_tags=["1*1e6/d*T", "RepCode Mean Positions" ], rep_delay=rep_delay) 


12:50:10 Warning: updating metadata. IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/libs/qiskit-saver-provider/result_saver/provider/saver_provider.py, LINE: 46
